In [27]:
import numpy as np 
import pandas as pd

In [28]:
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

C:\Users\lalith\AppData\Local\Temp\ipykernel_17832\3819754567.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('books.csv')


In [29]:
print(books.shape)
print(users.shape)
print(users.shape)
ratings.info()

(271360, 8)
(278858, 3)
(278858, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [30]:
#recommendation system

In [31]:
ratings_with_name = ratings.merge(books,on='ISBN')
ratings_with_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   User-ID              1031136 non-null  int64 
 1   ISBN                 1031136 non-null  object
 2   Book-Rating          1031136 non-null  int64 
 3   Book-Title           1031136 non-null  object
 4   Book-Author          1031134 non-null  object
 5   Year-Of-Publication  1031136 non-null  object
 6   Publisher            1031134 non-null  object
 7   Image-URL-S          1031136 non-null  object
 8   Image-URL-M          1031136 non-null  object
 9   Image-URL-L          1031132 non-null  object
dtypes: int64(2), object(8)
memory usage: 78.7+ MB


In [32]:
num_rating = ratings_with_name.groupby('Book-Title').agg(num_ratings=('Book-Rating','count')).reset_index()
num_rating

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [33]:
avg_rating = ratings_with_name.groupby('Book-Title').agg(avg_rating=('Book-Rating','mean')).reset_index()
avg_rating

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [34]:
popularity_df = num_rating.merge(avg_rating,on='Book-Title')
popularity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241071 entries, 0 to 241070
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Book-Title   241071 non-null  object 
 1   num_ratings  241071 non-null  int64  
 2   avg_rating   241071 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 5.5+ MB


In [35]:
popularity_df = popularity_df[popularity_df['num_ratings']>=200].sort_values('avg_rating',ascending=False)

In [36]:
popularity_df = popularity_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [37]:
temp = ratings_with_name.groupby('User-ID').count()['Book-Rating']>=200
good_users= temp[temp].index
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(good_users)]
filtered_rating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 475007 entries, 2 to 1030887
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   User-ID              475007 non-null  int64 
 1   ISBN                 475007 non-null  object
 2   Book-Rating          475007 non-null  int64 
 3   Book-Title           475007 non-null  object
 4   Book-Author          475006 non-null  object
 5   Year-Of-Publication  475007 non-null  object
 6   Publisher            475005 non-null  object
 7   Image-URL-S          475007 non-null  object
 8   Image-URL-M          475007 non-null  object
 9   Image-URL-L          475005 non-null  object
dtypes: int64(2), object(8)
memory usage: 39.9+ MB


In [38]:
temp = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
popular_books = temp[temp].index
final_df = filtered_rating[filtered_rating['Book-Title'].isin(popular_books)]

In [39]:
pt = final_df.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)

In [40]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
scores = cosine_similarity(pt)

In [43]:
def recommend(book_name):
    idx = np.where(pt.index==book_name)[0][0]
    items = sorted(list(enumerate(scores[idx])),key = lambda x:x[1],reverse=True)[1:6]
    data=[]
    for i in items:
        item=[]
        temp = books[books['Book-Title']==pt.idx[i[0]]]
        item.extend(list(temp.drop_duplicates('Book_Title')['Book-Title'].values))
        item.extend(list(temp.drop_duplicates('Book_Title')['Book-Author'].values))       
        item.extend(list(temp.drop_duplicates('Book_Title')['Image-URL-M'].values))
        data.append(item
    return data

SyntaxError: '(' was never closed (304364579.py, line 11)

In [ ]:
recommend('Harry Potter and the Prisoner of Azkaban (Book 3)')

In [ ]:
final_df.merge(popularity_df,on='Book-Title')
final_df.info()

In [ ]:
popularity_df.info()

In [ ]:
import pickle
pickle.dump(popularity_df,open('popularity_df.pkl','wb'))

In [ ]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(scores,open('scores.pkl','wb'))